In [9]:
import pandas as pd
import json

In [3]:
data = pd.read_json('business.json', lines=True)
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [7]:
sm = data.attributes[1]

In [12]:
sm.keys()

dict_keys(['RestaurantsReservations', 'GoodForMeal', 'BusinessParking', 'Caters', 'NoiseLevel', 'RestaurantsTableService', 'RestaurantsTakeOut', 'RestaurantsPriceRange2', 'OutdoorSeating', 'BikeParking', 'Ambience', 'HasTV', 'WiFi', 'GoodForKids', 'Alcohol', 'RestaurantsAttire', 'RestaurantsGoodForGroups', 'RestaurantsDelivery'])

In [25]:
import numpy as np

def get_keys(x):
    if x is not None:
        return np.array(list(x.keys()))
    else:
        return np.array([])

data["attribute_keys"] = data.attributes.apply(get_keys)

In [29]:
# Aggregate keys - These will likely need to be hardcoded as columns
unique_keys = set(np.concatenate(data.attribute_keys.to_numpy()))
unique_keys

{'AcceptsInsurance',
 'AgesAllowed',
 'Alcohol',
 'Ambience',
 'BYOB',
 'BYOBCorkage',
 'BestNights',
 'BikeParking',
 'BusinessAcceptsBitcoin',
 'BusinessAcceptsCreditCards',
 'BusinessParking',
 'ByAppointmentOnly',
 'Caters',
 'CoatCheck',
 'Corkage',
 'DietaryRestrictions',
 'DogsAllowed',
 'DriveThru',
 'GoodForDancing',
 'GoodForKids',
 'GoodForMeal',
 'HairSpecializesIn',
 'HappyHour',
 'HasTV',
 'Music',
 'NoiseLevel',
 'Open24Hours',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsCounterService',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTableService',
 'RestaurantsTakeOut',
 'Smoking',
 'WheelchairAccessible',
 'WiFi'}

In [ ]:
# Pickle unique keys for further use
import pickle
with open('keys.pkl', 'w+') as file:
    pickle.dumps(unique_keys, file)

In [67]:
# Turning Key: values into columns

subset = data.iloc[0:10]

def expand_attributes(df):
    attributes = {}
    default_na = {key: np.nan for key in unique_keys}
    if df.attributes is None:
        attributes = default_na
    else:
        for key in unique_keys:
            if key in df.attributes.keys():
                attributes[key] = df.attributes[key]
            else:
                attributes[key] = np.nan
    return attributes
        
attribute_df = pd.DataFrame.from_dict(
    dict(data.apply(expand_attributes, axis=1)),
    orient='index'
)

attribute_df.head()

,BYOB,RestaurantsReservations,Smoking,RestaurantsAttire,CoatCheck,BikeParking,BYOBCorkage,RestaurantsDelivery,Ambience,OutdoorSeating,...,Corkage,DietaryRestrictions,Alcohol,BusinessParking,NoiseLevel,AcceptsInsurance,GoodForMeal,BusinessAcceptsBitcoin,HairSpecializesIn,WiFi
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,True,NaN,u'casual',NaN,False,NaN,False,"{'romantic': False, 'intimate': False, 'classy...",False,...,NaN,NaN,u'full_bar',"{'garage': False, 'street': False, 'validated'...",u'loud',NaN,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,u'no'
2,NaN,True,NaN,'casual',NaN,True,NaN,False,"{'romantic': False, 'intimate': False, 'touris...",False,...,NaN,NaN,u'beer_and_wine',"{'garage': False, 'street': False, 'validated'...",u'average',NaN,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,u'no'
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN


In [80]:
# Clean Individual attributes
# Remove non ascii chars from string
def clean_strings(x):
    if "u'" in str(x):
        return x.strip("u'")
    else:
        return x

for key in unique_keys:
    attribute_df[key] = attribute_df[key].apply(clean_strings)
    
attribute_df.head()

,BYOB,RestaurantsReservations,Smoking,RestaurantsAttire,CoatCheck,BikeParking,BYOBCorkage,RestaurantsDelivery,Ambience,OutdoorSeating,...,Corkage,DietaryRestrictions,Alcohol,BusinessParking,NoiseLevel,AcceptsInsurance,GoodForMeal,BusinessAcceptsBitcoin,HairSpecializesIn,WiFi
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,True,NaN,casual,NaN,False,NaN,False,"{'romantic': False, 'intimate': False, 'classy...",False,...,NaN,NaN,full_bar,"{'garage': False, 'street': False, 'validated'...",loud,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,no
2,NaN,True,NaN,'casual',NaN,True,NaN,False,"{'romantic': False, 'intimate': False, 'touris...",False,...,NaN,NaN,beer_and_wine,"{'garage': False, 'street': False, 'validated'...",average,NaN,"{'dessert': False, 'latenight': False, 'lunch'...",NaN,NaN,no
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN


In [88]:
# Try concatenating with main data
frame = [data, attribute_df]
new_data = pd.concat(frame, axis=1, sort=True, join='inner')
new_data.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Corkage,DietaryRestrictions,Alcohol,BusinessParking,NoiseLevel,AcceptsInsurance,GoodForMeal,BusinessAcceptsBitcoin,HairSpecializesIn,WiFi
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# How to get attributes into database

There are a couple ways to approach this.  One is to continue cleaning and condense back to a uniform size attributes list.  Another would be to migrate the business table to something with all these features. I don't think the migration is worthwhile.  The raw data isn't done as such, and the possibity of new feaures downstream means another migration may be needed.

Few Steps:

1. Convert dictionary items to simple lists of True.  (Don't include false items).
2. Condense all attribute columns (from attribute_df) into single dictionary of lists (or nan).